##SETUP

In [4]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'importlib' not in sys.modules:  
    start_scrcpy=True
    if start_scrcpy:
        proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
        time.sleep(6) # <-- sleep for 6''
        proc.terminate() # <-- terminate the process
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1


bot = bot_core.Bot()

In [19]:
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

## Game stats

In [159]:
# get stored mana 
#bot.getText(220,1360,90,50,new=False,digits=True)

# get unit mana 
bot.getText(450,1360,90,50,new=False,digits=True)

# Get upgrade mana
#bot.getText(20,1540,700,50,new=False,digits=True).split('6')


'70'

In [3]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()
#cache_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,empty.png,999.0,0,0.0,"[0, 0]",1.0
1,icon_1,alchemist.png,700.0,1,16184.5,"[0, 1]",1.0
2,icon_2,sharpshooter.png,335.0,2,940.0,"[0, 2]",1.0
3,icon_3,alchemist.png,228.0,1,1577.5,"[0, 3]",1.0
4,icon_4,vampire.png,187.0,1,1459.5,"[0, 4]",1.0
5,icon_5,empty.png,999.0,0,0.0,"[1, 0]",1.0
6,icon_6,empty.png,999.0,0,0.0,"[1, 1]",1.0
7,icon_7,vampire.png,179.0,1,4235.5,"[1, 2]",1.0
8,icon_8,lightning.png,190.0,1,2373.0,"[1, 3]",1.0
9,icon_9,lightning.png,178.0,1,1476.5,"[1, 4]",1.0


In [6]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
#unit_series.drop(labels='empty.png',level=0)
merge_series = unit_series[unit_series>2]
merge_target =  merge_series.sample().index[0]
merge_df=df_split.get_group(merge_target)
merge_df.sample(n=2)

,grid_id,unit,probability,rank,rank_error,position,Age
7,icon_7,crystal.png,163.0,1,1631.0,"[1, 2]",5.0
9,icon_9,crystal.png,216.0,1,1632.5,"[1, 4]",5.0


In [110]:
button_df = bot.get_current_icons()
avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
avail_buttons

,icon,available,"pos [X,Y]"


In [8]:
bot_core.adv_filter_keys(unit_series,[
                            [3,4,5],
                            ['crystal.png']
                            ],remove=True)

unit         rank
chemist.png  1       3
crystal.png  1       3
empty.png    0       3
hunter.png   1       3
crystal.png  2       1
hunter.png   3       1
monkey.png   2       1
Name: unit, dtype: int64

# Play Game

In [67]:
bot.getScreen()

In [3]:
bot.restart_RR()
# Drop co-op game
#for i in range(30):
#    bot.restart_RR(True)

## Extra unit logic 

In [41]:
##Extra unit logic 
# Try to merge any priestess
#if 'priest.png' in unit_list:
#    if df_groups['priest.png']<1:
#        bot.merge_unit(grid_df,'priest.png')

# Basic Loop

In [11]:
def try_merge(rank=1,prev_grid=None):
    info=''
    merge_df =None
    names=bot.scan_grid(new=True)
    grid_df=bot_perception.grid_status(names,prev_grid=prev_grid)
    df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
    # Select stuff to merge
    merge_series = unit_series[unit_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
    # check if grid full
    if ('empty.png',0) in group_keys:
        # Try to merge priest
        merge_priest = bot_core.filter_keys(merge_series,['priest.png'])
        if not merge_priest.empty:
            merge_df = bot.merge_unit(df_split,merge_priest)
        # Merge if full board
        if df_groups['empty.png']<=2:
            info='Merging!'
            # Add criteria
            merge_series = bot_core.filter_keys(merge_series,[rank,'priest.png'])
            if not merge_series.empty:
                # drop duplicated indices
                merge_series = merge_series[~merge_series.index.duplicated()]
                # Take index name of random sample
                merge_target =  merge_series.sample().index[0] 
                # Retrieve group    
                merge_df=df_split.get_group(merge_target)
                #merge_df=merge_df.sort_values(by='Age',ascending=False).reset_index(drop=True)
                # Send merge command
                merge_df = bot.merge_unit(df_split,merge_series)
            else:
                info='not enough filtered targets!'
        else: info= 'need more units!'
    # If grid seems full, merge any
    else:
        info = 'Full Grid - Merging!'
        # Remove all high level crystals

        merge_df = bot.merge_unit(df_split,merge_series)
    return grid_df,unit_series,merge_df,info
    
def get_button_pos(df,button):
    #button=button+'.png'
    pos=df[df['icon']==button]['pos [X,Y]'].reset_index(drop=True)[0]
    return np.array(pos)
# Mana level cards
def mana_level(cards, hero_power=False):
    upgrade_pos_dict={
    1:[100,1500], 2:[200,1500],
    3:[350,1500], 4:[500,1500],
    5:[650,1500] }
    # Level each card
    for card in cards:
        bot.click(*upgrade_pos_dict[card])
    if hero_power:
        bot.click(800,1500)
# Start a dungeon floor from PvE page
def play_dungeon(floor=5):
    # Divide by 3 and take ceiling of floor as int
    target_chapter = f'chapter_{int(np.ceil(floor/3))}.png'
    expanded=0
    avail_buttons = bot.get_current_icons(available=True)
    # Check if on dungeon page
    if (avail_buttons == 'dungeon_page.png').any(axis=None):
        # Swipe to the top
        [bot.swipe([0,0],[2,0]) for i in range(10)]
        bot.click(30,600,5) # stop scroll and scan screen for buttons
        # Keep swiping until floor is found
        for i in range(10):
            avail_buttons = bot.get_current_icons(available=True)
            # Look for correct chapter
            if (avail_buttons == target_chapter).any(axis=None):
                pos = get_button_pos(avail_buttons,target_chapter)
                if not expanded:
                    expanded = 1
                    bot.click_button(pos+[480,90])
                # check button is near top of screen
                if pos[1] < 550:
                    # Stop scrolling
                    break
            # Swipe down (change to swipe up for floor 10 cleared)
            [bot.swipe([2,0],[0,0]) for i in range(1)]
            bot.click(30,600) # stop scroll and scan screen for buttons
        # Maybe add check if it worked? alt just check for fight page
        ## Click play floor
        bot.click_button(pos+[0,85+400*(floor%3)]) #(only 1,2, 4,5, 7, 8 possible)
        bot.click_button((130,950))        

# Locate game home screen and try to start fight is chosen
def battle_screen(start=False,pve=True,floor=5):
    # Scan screen for any key buttons
    button_df = bot.get_current_icons()
    # filter relevant buttons
    button_df = button_df[button_df['icon'].isin(['back_button.png','battle_icon.png','0cont_button.png','1quit.png', 'fighting.png','pvp_button.png'])]
    avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
    if not avail_buttons.empty:
        # list of buttons
        button_names=avail_buttons['icon'].tolist()
        if 'fighting.png' in button_names and not '0cont_button.png' in button_names:
            return avail_buttons,'fighting'
        # Start pvp if homescreen
        if start and 'pvp_button.png' in button_names and 'battle_icon.png' in button_names:
            pvp_pos = get_button_pos(button_df,'pvp_button.png')
            if pve:
                # Add a 500 pixel offset for PvE button
                bot.click_button(pvp_pos+[500,0])
                play_dungeon(floor=floor)
            else: bot.click_button(pvp_pos)
            time.sleep(1)
            return avail_buttons, 'home'
        # Check first button is clickable
        first_button=button_names[0].split('.')[0]
        if first_button in ['back_button','battle_icon','0cont_button','1quit']:
            button_pos=avail_buttons['pos [X,Y]'].tolist()[0]
            bot.click_button(button_pos)
            return avail_buttons,'menu'
    else:
        bot.shell(f'input keyevent {const.KEYCODE_BACK}') #Force back
        return button_df,'lost'

battle_screen() 

(           icon  available    pos [X,Y]
 0  fighting.png       True  (669, 1359),
 'fighting')

In [50]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
# Select stuff to merge
merge_series = unit_series[unit_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
# check if grid full
if ('empty.png',0) in group_keys:
    # Try to merge priest
    merge_priest = bot_core.filter_keys(merge_series,['priest.png'])

In [13]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
# Select stuff to merge
merge_series = unit_series[unit_series>=2] # at least 2 units 
merge_series

unit           rank
lightning.png  1       3
alchemist.png  1       2
empty.png      0       2
priest.png     2       2
Name: unit, dtype: int64

## RUN BOT

In [12]:
wait=0
combat = 0
grid_df =None
# Main loop
while(1):
    output = battle_screen(True,True,5)
    if output[1]=='fighting':
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                mana_level([3,1,5],hero_power=True)
                # Try to merge units
                grid_df,unit_series,df_groups,info = try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 '+output[1] +'  '+info,grid_df,unit_series[0:5],df_groups])
    else:
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            wait=0
        print(output[1],'wait count:',wait)

['29, 3/8 fighting  Merging!',
     grid_id         unit  probability  rank  rank_error position   Age
 0    icon_0  crystal.png        308.0     3       576.0   [0, 0]   0.0
 1    icon_1   monkey.png        289.0     2       630.0   [0, 1]  21.0
 2    icon_2   monkey.png        208.0     1      1208.0   [0, 2]  11.0
 3    icon_3  chemist.png        210.0     1      1333.0   [0, 3]   2.0
 4    icon_4   monkey.png        299.0     4       544.0   [0, 4]  16.0
 5    icon_5  crystal.png        214.0     2       575.0   [1, 0]  19.0
 6    icon_6    empty.png        700.0     0         0.0   [1, 1]   5.0
 7    icon_7  chemist.png        189.0     1      1151.0   [1, 2]   0.0
 8    icon_8   hunter.png        232.0     1      3242.5   [1, 3]  14.0
 9    icon_9  crystal.png        344.0     4       311.0   [1, 4]  10.0
 10  icon_10    dr5ad.png        673.0     5       442.5   [2, 0]   0.0
 11  icon_11  chemist.png        404.0     1      2972.5   [2, 1]   2.0
 12  icon_12  crystal.png        

ValueError: a must be greater than 0 unless no samples are taken